## 문자-단위 RNN 으로 이름 생성

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

In [3]:
all_letters = string.ascii_letters + ".,;'-"
n_letters = len(all_letters) + 1 # EOS  기호를 추가하여 갯수를 센다

In [ ]:
def findFiles(path): return glob.glob(path)

def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
  if unicodedata.category(c) != 'Mn'
  and c in all_letters
  )